In [2]:
import json
import folium 
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [3]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [4]:
covid1 = json.loads(data)

In [6]:
#normalise data
pd.json_normalize(covid1['Countries'], sep=',')

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,21,39254,3,1458,104,32746,2020-09-30T10:54:19Z
1,Albania,AL,albania,127,13518,4,384,103,7732,2020-09-30T10:54:19Z
2,Algeria,DZ,algeria,155,51368,7,1726,101,36063,2020-09-30T10:54:19Z
3,Andorra,AD,andorra,0,1966,0,53,0,1265,2020-09-30T10:54:19Z
4,Angola,AO,angola,108,4905,3,179,20,1833,2020-09-30T10:54:19Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,17,1094,0,35,8,1007,2020-09-30T10:54:19Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-30T10:54:19Z
185,Yemen,YE,yemen,0,2031,0,587,9,1275,2020-09-30T10:54:19Z
186,Zambia,ZM,zambia,55,14715,0,332,116,13937,2020-09-30T10:54:19Z


In [7]:
#convert to pandas dataframe

df = pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,21,39254,3,1458,104,32746,2020-09-30T10:54:19Z,{}
1,Albania,AL,albania,127,13518,4,384,103,7732,2020-09-30T10:54:19Z,{}
2,Algeria,DZ,algeria,155,51368,7,1726,101,36063,2020-09-30T10:54:19Z,{}
3,Andorra,AD,andorra,0,1966,0,53,0,1265,2020-09-30T10:54:19Z,{}
4,Angola,AO,angola,108,4905,3,179,20,1833,2020-09-30T10:54:19Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,17,1094,0,35,8,1007,2020-09-30T10:54:19Z,{}
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-30T10:54:19Z,{}
185,Yemen,YE,yemen,0,2031,0,587,9,1275,2020-09-30T10:54:19Z,{}
186,Zambia,ZM,zambia,55,14715,0,332,116,13937,2020-09-30T10:54:19Z,{}


In [8]:
#drop unwanted cols
covid = df.drop(columns=['CountryCode', 'Slug', 'Date', 'Premium'], axis=1)
covid

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,21,39254,3,1458,104,32746
1,Albania,127,13518,4,384,103,7732
2,Algeria,155,51368,7,1726,101,36063
3,Andorra,0,1966,0,53,0,1265
4,Angola,108,4905,3,179,20,1833
...,...,...,...,...,...,...,...
183,Viet Nam,17,1094,0,35,8,1007
184,Western Sahara,0,10,0,1,0,8
185,Yemen,0,2031,0,587,9,1275
186,Zambia,55,14715,0,332,116,13937


In [9]:
map1 = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
map1

&lt;folium.folium.Map at 0x209b3b38100&gt;

In [10]:
url = 'http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [11]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid,
    columns=['Country', 'TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color='OrRd',
    nana_fill_color='black',
    legend_name='Total Confirmed Covid-19 Cases',
).add_to(map1)
map1

&lt;folium.folium.Map at 0x209b3b38100&gt;

In [12]:
covid.update(covid['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid.update(covid['TotalRecovered'].map('Total Recovered:{}'.format))

In [13]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [14]:
covid_final=pd.merge(covid, coordinates, on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
    radius=5,
    weight=2,
    popup = [point.Country, point.TotalConfirmed, point.TotalRecovered],
    fill_color = '#000000').add_to(map1)

In [17]:
covid_final.apply(plotDot, axis=1)
map1.fit_bounds(map1.get_bounds())
map1

&lt;folium.folium.Map at 0x209b3b38100&gt;

In [31]:
map1 = folium.Map(tiles='StamenToner', min_zoom=2)
map1

&lt;folium.folium.Map at 0x209bad52e20&gt;

In [36]:
deaths = covid_final['TotalDeaths'].astype(float)
lat = covid_final['latitude'].astype(float)
lon = covid_final['longitude'].astype(float)
map1.add_child(HeatMap(zip(lat,lon,deaths), radius=0)) 

&lt;folium.folium.Map at 0x209bad52e20&gt;

In [35]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
    radiu=2,
    weight=1,
    popup=[point.Country, point.TotalDeaths],
    fill_color='#000000').add_to(map1)
covid_final.apply(plotDot, axis=1)
map1.fit_bounds(map1.get_bounds())
map1

&lt;folium.folium.Map at 0x209bad52e20&gt;